# SA_SLM 학습 노트북

## 필요 파일
- `data.zip`: 로컬 `data/` 폴더를 ZIP 압축

## 실행 전
런타임 > 런타임 유형 변경 > **GPU (T4)** 선택

In [ ]:
# 1. 환경 설정
%pip install -q torch transformers datasets accelerate peft bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 51.0 MB/s eta 0:00:00


In [ ]:
# 2. 데이터 업로드
from google.colab import files
import zipfile, os, json, glob

print('data.zip 파일을 업로드하세요')
uploaded = files.upload()

for f in uploaded:
    if f.endswith('.zip'):
        zipfile.ZipFile(f).extractall('.')
        print(f'{f} 압축 해제 완료')

data.zip 파일을 업로드하세요


Saving data.zip to data.zip
data.zip 압축 해제 완료


In [ ]:
# 3. 데이터 로드
records = [json.load(open(f)) for f in
           glob.glob('./data/examples/student_record*.json') +
           glob.glob('./data/raw/students/*.json')]

tasks = {}
for k, v in [('roadmap', 'test_task1_roadmap.json'),
             ('statement', 'test_task2_statement.json'),
             ('evaluation', 'test_task3_evaluation.json')]:
    p = './data/examples/' + v
    if os.path.exists(p):
        tasks[k] = json.load(open(p))

print(f'{len(records)} records, {sum(len(v) for v in tasks.values())} tasks')

101 records, 20 tasks


In [ ]:
# 4. 학습 데이터 변환 (Schema v2)
examples = []

for r in records:
    # 신 스키마 v2 구조
    school = r.get('school_info', {})
    grades = r.get('grades', {})
    major = r.get('target_major', {})
    acts_obj = r.get('activities', {})
    notes = r.get('consultant_notes', {})
    result = r.get('admission_result', {})

    # 교과 + 비교과 활동 통합
    all_acts = acts_obj.get('curricular', []) + acts_obj.get('extracurricular', [])

    # 프로필 문자열 생성
    pstr = f"지역:{school.get('region','')} 학교:{school.get('school_type','')} 계열:{major.get('track','')} 성적:{grades.get('overall_tier','')} 관심:{','.join(major.get('interests',[]))} 가치관:{','.join(major.get('values',[]))} 목표:{major.get('specific','')}"

    # 차별화된 활동 (점수 4 이상)
    astr = ''.join([f"\n-[{a.get('category','')}]{a.get('description','')}" for a in all_acts if a.get('uniqueness_score',0)>=4])

    # 활동 추천 예시
    examples.append({'i': '활동추천', 'in': pstr, 'out': f"[추천]{astr}\n[서사]{notes.get('narrative_summary','')}"})

    # 세특 작성 예시
    for a in all_acts:
        if a.get('neis_statement'):
            examples.append({'i': '세특작성', 'in': f"과목:{a.get('subject','')} 활동:{a.get('description','')}", 'out': a.get('neis_statement','')})

# 기존 task 파일 처리 (하위 호환)
for item in tasks.get('roadmap', []):
    inp = item.get('input', {})
    examples.append({'i': item.get('instruction',''), 'in': f"계열:{inp.get('track','')}", 'out': item.get('output','')})

for item in tasks.get('statement', []):
    inp = item.get('input', {})
    examples.append({'i': item.get('instruction',''), 'in': f"과목:{inp.get('subject','')} 활동:{inp.get('raw_activity','')}", 'out': item.get('output','')})

print(f'{len(examples)} examples')

721 examples


In [ ]:
# 5. Dataset 생성 (SFTTrainer용)
from datasets import Dataset

# SFTTrainer는 텍스트 컬럼만 필요로 함
def fmt(e):
    return {'text': f"<|im_start|>system\n생기부전문가<|im_end|>\n<|im_start|>user\n{e['i']}\n{e['in']}<|im_end|>\n<|im_start|>assistant\n{e['out']}<|im_end|>"}

dataset = Dataset.from_list([fmt(e) for e in examples])
print(f'{len(dataset)} samples ready for SFTTrainer')

721 samples ready for SFTTrainer


In [ ]:
# 6. 모델 로드 + LoRA 설정
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training

MODEL = 'Qwen/Qwen2.5-3B-Instruct'

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL, quantization_config=bnb, device_map='auto', trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16, lora_alpha=32,
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj'],
    lora_dropout=0.05, bias='none', task_type='CAUSAL_LM'
)

print('Model loaded!')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Model loaded!


In [12]:
# 7. 학습 (SFTTrainer - 최소 버전)
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./out',
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    bf16=True,
    logging_steps=10,
    save_strategy='epoch',
    optim='adamw_torch',
)

# formatting_func으로 데이터 포맷 전달
def formatting_func(example):
    # train_dataset (dataset)은 이미 {'text': '...'} 형식으로 포맷되어 있습니다.
    # SFTTrainer의 formatting_func는 텍스트 문자열을 반환해야 합니다.
    return example['text']

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    args=training_args,
    formatting_func=formatting_func,
    # `tokenizer` 인자가 현재 trl 버전에서 예상되지 않아 제거합니다.
    # 하지만 이로 인해 학습 과정에서 토크나이저를 찾지 못하는 오류가 발생할 수 있습니다.
    # 이 경우 trl 및 transformers 라이브러리 버전을 확인하고 재설치하는 것이 좋습니다.
)

print(f'Training {len(dataset)} samples...')
trainer.train()

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/721 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/721 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/721 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/721 [00:00<?, ? examples/s]

...

wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.556300
20,1.106200
30,0.799100
40,0.488800
50,0.296500
60,0.313600
70,0.313600
80,0.258000
90,0.302500
100,0.191500


wandb: WARNING URL not available in offline run
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
wandb: WARNING URL not available in offline run
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more deta

TrainOutput(global_step=1810, training_loss=0.14436131406225552, metrics={'train_runtime': 2334.6215, 'train_samples_per_second': 3.088, 'train_steps_per_second': 0.775, 'total_flos': 1.6847966669930496e+16, 'train_loss': 0.14436131406225552})

In [14]:
# 8. LoRA Adapter만 저장 & 다운로드
import shutil

# LoRA adapter만 저장 (base model 제외)
model.save_pretrained('./lora_adapter', save_adapter=True, save_config=True)
tokenizer.save_pretrained('./lora_adapter')

# ZIP 압축
shutil.make_archive('sa_slm_lora_adapter', 'zip', '.', './lora_adapter')

print('LoRA adapter 저장 완료!')
print('예상 크기: 50~100MB')

# 다운로드
files.download('sa_slm_lora_adapter.zip')

/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3867: UserWarning: `save_config` is deprecated and will be removed in v5 of Transformers. Use `is_main_process` instead.
  warnings.warn(


LoRA adapter 저장 완료!
예상 크기: 50~100MB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
# LoRA adapter만 추출 (올바른 방법)
from peft import get_peft_model_state_dict
import torch
import json
import shutil
import os

# adapter 폴더 생성
os.makedirs('./lora_only', exist_ok=True)

# LoRA 가중치만 추출
adapter_state = get_peft_model_state_dict(model)
torch.save(adapter_state, './lora_only/adapter_model.bin')

# adapter config 저장
model.peft_config['default'].save_pretrained('./lora_only')

# tokenizer 저장
tokenizer.save_pretrained('./lora_only')

# ZIP 압축 & 다운로드
shutil.make_archive('sa_slm_lora_only', 'zip', '.', './lora_only')
files.download('sa_slm_lora_only.zip')

print('완료! 예상 크기: 30~80MB')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

완료! 예상 크기: 30~80MB
